In [1]:
import numpy as np
import random
import scipy.integrate as scp
import numpy.random as rnd
import time
import matplotlib.pyplot as plt
import numpy.random as rnd
import copy
from matplotlib.pyplot import figure
from pyomo.environ import *
from pyomo.dae import *
import pandas as pd
import pickle
import scipy.stats
from math import comb
from numpy.polynomial import polynomial as P
from scipy.integrate import odeint
from pyomo.environ import ConstraintList

eps  = np.finfo(float).eps

In [2]:

''' Data treatment'''
def save_pkl(item, fname):
    sn = 'tmp7/' + fname
    with open(sn, 'wb') as handle:
        pickle.dump(item, handle) #, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'File saved at: {sn}')
    return None
# return None

def load_pkl(fname):
    with open(fname, 'rb') as handle:
        ans = pickle.load(handle)
    print(f'Loaded from: {fname}')
    return ans

def to_dict(x,dt):
    obs = list(x)
    # tp = list(time)
    dicx = {}
    for i in range(len(x)):
        dicx[dt*i] = obs[i]            # change thye value of 38.4
    return dicx

def get_grad(x, t):
    dxdt = [[],[],[],[]]
    for n in range(x.shape[0]):
        for i in range(len(x[0]) - 1):
            dxdt[n].append((x[n][i + 1] - x[n][i])/(t[i + 1] - t[i]))
        dxdt[n].append(dxdt[n][-1])
    return dxdt


In [3]:
# eff_neuron = 7# actually applied number of neurons
no_euron = 7 # total number of neurons

In [4]:
xobs1 = load_pkl('data/xobs1.pkl')
xobs2 = load_pkl('data/xobs2.pkl')
xobs3 = load_pkl('data/xobs3.pkl')
xobs4 = load_pkl('data/xobs4.pkl')
mu_con   = load_pkl('data/u_con.pkl')
tf_N  = load_pkl('data/tf_N.pkl')
N_in  = load_pkl('data/N_in.pkl')
operation_con = load_pkl('data/operation_con.pkl')


tt1 = load_pkl('data/tt1.pkl')
std_value1 = load_pkl('data/std_value1.pkl')
std_value2 = load_pkl('data/std_value2.pkl')
std_value3 = load_pkl('data/std_value3.pkl')
std_value4 = load_pkl('data/std_value4.pkl')
data_init = load_pkl('data/data_init.pkl')

operation_con1 = operation_con[0]
operation_con2 = operation_con[1]
operation_con3 = operation_con[2]
operation_con4 = operation_con[3]

mu1 = mu_con[0] #u contains FNin and I0
mu2 = mu_con[1] 
mu3 = mu_con[2] 
mu4 = mu_con[3] 

tf    = 240
# tf    = 16*24
steps_= 10
dt    = tf/steps_ #dt is fixed





Loaded from: data/xobs1.pkl
Loaded from: data/xobs2.pkl
Loaded from: data/xobs3.pkl
Loaded from: data/xobs4.pkl
Loaded from: data/u_con.pkl
Loaded from: data/tf_N.pkl
Loaded from: data/N_in.pkl
Loaded from: data/operation_con.pkl
Loaded from: data/tt1.pkl
Loaded from: data/std_value1.pkl
Loaded from: data/std_value2.pkl
Loaded from: data/std_value3.pkl
Loaded from: data/std_value4.pkl
Loaded from: data/data_init.pkl


In [5]:
mu_con

[array([[0.00e+00, 1.00e+02],
        [2.50e-03, 1.50e+02],
        [3.75e-03, 1.25e+02],
        [1.25e-03, 1.75e+02]]),
 array([[1.875e-03, 1.375e+02],
        [4.375e-03, 1.875e+02],
        [3.125e-03, 1.125e+02],
        [6.250e-04, 1.625e+02]]),
 array([[9.3750e-04, 1.3125e+02],
        [3.4375e-03, 1.8125e+02],
        [4.6875e-03, 1.0625e+02],
        [2.1875e-03, 1.5625e+02]]),
 array([[1.5625e-03, 1.1875e+02],
        [4.0625e-03, 1.6875e+02],
        [2.8125e-03, 1.4375e+02],
        [3.1250e-04, 1.9375e+02]])]

In [6]:
x_max = np.max(np.array([xobs1[0],xobs2[0],xobs3[0],xobs4[0]]))
q_max = np.max(np.array([xobs1[2],xobs2[2],xobs3[2],xobs4[2]]))
I_max = np.max(np.concatenate(mu_con).T[1])

In [7]:
I_max 

193.75

In [8]:
N_in

100

In [9]:
xobs1.shape

(5, 11)

In [22]:
mu1[0][1]

array([[0.00e+00, 1.00e+02],
       [2.50e-03, 1.50e+02],
       [3.75e-03, 1.25e+02],
       [1.25e-03, 1.75e+02]])

In [10]:

num_N = 4
tf_N  = tf/(num_N)
dstep_N = int(tf_N/dt)

def get_grad(x, t):
    dxdt = [[],[],[],[],[]]
    for n in range(x.shape[0]):
        for i in range(len(x[0]) - 1):
            dxdt[n].append((x[n][i + 1] - x[n][i])/(t[i + 1] - t[i]))
        dxdt[n].append(dxdt[n][-1])
    return dxdt

Xt1   = [to_dict(xobs1[0],dt), to_dict(xobs1[1],dt), to_dict(xobs1[2],dt), to_dict(xobs1[3],dt), to_dict(xobs1[4],dt)] 
dXdt1 = [to_dict(get_grad(xobs1, tt1)[0],dt),to_dict(get_grad(xobs1, tt1)[1],dt),to_dict(get_grad(xobs1, tt1)[2],dt),to_dict(get_grad(xobs1, tt1)[3],dt),to_dict(get_grad(xobs1, tt1)[4],dt)]
Xt2   = [to_dict(xobs2[0],dt), to_dict(xobs2[1],dt), to_dict(xobs2[2],dt), to_dict(xobs2[3],dt), to_dict(xobs2[4],dt)] 
dXdt2 = [to_dict(get_grad(xobs2, tt1)[0],dt),to_dict(get_grad(xobs2, tt1)[1],dt),to_dict(get_grad(xobs2, tt1)[2],dt),to_dict(get_grad(xobs2, tt1)[3],dt),to_dict(get_grad(xobs2, tt1)[4],dt)]
Xt3   = [to_dict(xobs3[0],dt), to_dict(xobs3[1],dt), to_dict(xobs3[2],dt), to_dict(xobs3[3],dt), to_dict(xobs3[4],dt)] 
dXdt3 = [to_dict(get_grad(xobs3, tt1)[0],dt),to_dict(get_grad(xobs3, tt1)[1],dt),to_dict(get_grad(xobs3, tt1)[2],dt),to_dict(get_grad(xobs3, tt1)[3],dt),to_dict(get_grad(xobs3, tt1)[4],dt)]
Xt4   = [to_dict(xobs4[0],dt), to_dict(xobs4[1],dt), to_dict(xobs4[2],dt), to_dict(xobs4[3],dt), to_dict(xobs4[4],dt)] 
dXdt4 = [to_dict(get_grad(xobs4, tt1)[0],dt),to_dict(get_grad(xobs4, tt1)[1],dt),to_dict(get_grad(xobs4, tt1)[2],dt),to_dict(get_grad(xobs4, tt1)[3],dt),to_dict(get_grad(xobs4, tt1)[4],dt)]
# Xt5   = [to_dict(xobs5[0],dt), to_dict(xobs5[1],dt), to_dict(xobs5[2],dt), to_dict(xobs5[3],dt)] 
# dXdt5 = [to_dict(get_grad(xobs5, tt1)[0],dt),to_dict(get_grad(xobs5, tt1)[1],dt),to_dict(get_grad(xobs5, tt1)[2],dt),to_dict(get_grad(xobs5, tt1)[3],dt)]



In [11]:
Xt1[4]

{0.0: 0.5,
 24.0: 0.5,
 48.0: 0.5,
 72.0: 0.5299999999999992,
 96.0: 0.5899999999999989,
 120.0: 0.6499999999999979,
 144.0: 0.739999999999999,
 168.0: 0.829999999999999,
 192.0: 0.8899999999999982,
 216.0: 0.9199999999999973,
 240.0: 0.9499999999999974}

In [12]:
N1 = operation_con[0]
N2 = operation_con[1]
N3 = operation_con[2]
N4 = operation_con[3]

In [16]:
mu1[0]

array([  0., 100.])

In [33]:
Xt1

[{0.0: 0.18,
  24.0: 0.5788791079922703,
  48.0: 0.8752425776444962,
  72.0: 1.0945883131292757,
  96.0: 1.2471987458761729,
  120.0: 1.1714276107187842,
  144.0: 1.1280349920412818,
  168.0: 1.0868458644603565,
  192.0: 1.2873534882601663,
  216.0: 1.3685201162402763,
  240.0: 1.3910202506917433},
 {0.0: 15.0,
  24.0: 0.05454026931257367,
  48.0: 1e-07,
  72.0: 0.1649582480981474,
  96.0: 0.1606680984430304,
  120.0: 0.14494494265487884,
  144.0: 0.13654041601168823,
  168.0: 0.12212177564194876,
  192.0: 0.02050647012080513,
  216.0: 0.04541928754194248,
  240.0: 0.05010396747357495},
 {0.0: 80.0,
  24.0: 52.33206661360817,
  48.0: 33.52436146847306,
  72.0: 24.73793676681979,
  96.0: 25.00117757844649,
  120.0: 24.729564194713546,
  144.0: 23.08715917358975,
  168.0: 24.995128713942247,
  192.0: 22.204610130141642,
  216.0: 26.280382587433675,
  240.0: 22.300916857155606},
 {0.0: 120.0,
  24.0: 300.33072883879265,
  48.0: 417.72162151873374,
  72.0: 391.09598189874197,
  96.0: 352.1

In [34]:
model         = AbstractModel()

# -- variable definition -- #

# defining time as continous variable
model.t       = ContinuousSet(bounds=[0, tt1[-1]])


# defining measurement times
model.tm      = Set(within=model.t)



# defining measured values as parameters
model.x1_noise = Param(model.tm)
model.n1_noise = Param(model.tm)
model.q1_noise = Param(model.tm)
model.f1_noise = Param(model.tm)

# model.x2_noise = Param(model.tm)
# model.n2_noise = Param(model.tm)
# model.q2_noise = Param(model.tm)
# model.f2_noise = Param(model.tm)

# model.x3_noise = Param(model.tm)
# model.n3_noise = Param(model.tm)
# model.q3_noise = Param(model.tm)
# model.f3_noise = Param(model.tm)


# model.x4_noise = Param(model.tm)
# model.n4_noise = Param(model.tm)
# model.q4_noise = Param(model.tm)
# model.f4_noise = Param(model.tm)

# model.x5_noise = Param(model.tm)
# model.n5_noise = Param(model.tm)
# model.q5_noise = Param(model.tm)
# model.f5_noise = Param(model.tm)
# defining state variables
model.x1 = Var(model.t, within=PositiveReals,initialize=Xt1[0]) 
model.n1 = Var(model.t, within=PositiveReals,initialize=Xt1[1])
model.q1 = Var(model.t, within=PositiveReals,initialize=Xt1[2])
model.f1 = Var(model.t, within=PositiveReals,initialize=Xt1[3]) 

# model.V1 = Var(model.t, within=PositiveReals,initialize=Xt1[4])  

# model.x2 = Var(model.t, within=PositiveReals,initialize=Xt2[0]) 
# model.n2 = Var(model.t, within=PositiveReals,initialize=Xt2[1])
# model.q2 = Var(model.t, within=PositiveReals,initialize=Xt2[2])
# model.f2 = Var(model.t, within=PositiveReals,initialize=Xt2[3]) 

# model.x3 = Var(model.t, within=PositiveReals,initialize=Xt3[0]) 
# model.n3 = Var(model.t, within=PositiveReals,initialize=Xt3[1])
# model.q3 = Var(model.t, within=PositiveReals,initialize=Xt3[2])
# model.f3 = Var(model.t, within=PositiveReals,initialize=Xt3[3])

# model.x4 = Var(model.t, within=PositiveReals,initialize=Xt4[0]) 
# model.n4 = Var(model.t, within=PositiveReals,initialize=Xt4[1])
# model.q4 = Var(model.t, within=PositiveReals,initialize=Xt4[2])
# model.f4 = Var(model.t, within=PositiveReals,initialize=Xt4[3])

# model.F_in1 = Var(model.t, within=NonNegativeReals,initialize=float(mu1[0][0]))
# model.Fcn2 = Var(model.t, within=NonNegativeReals,initialize=float(u2[0][0]))
# model.Fcn3 = Var(model.t, within=NonNegativeReals,initialize=float(u3[0][0]))
# model.Fcn4 = Var(model.t, within=NonNegativeReals,initialize=float(u4[0][0]))

# model.I1 = Var(model.t, within=NonNegativeReals,initialize=float(mu1[0][1]))
model.I1 = Param(initialize=100)
# model.F_in1 =Param(initialize=0.000001)
# model.N_in = Param(initialize=100)
# model.I2 = Var(model.t, within=NonNegativeReals,initialize=float(u2[0][1]))
# model.I3 = Var(model.t, within=NonNegativeReals,initialize=float(u3[0][1]))
# model.I4 = Var(model.t, within=NonNegativeReals,initialize=float(u4[0][1]))
# model.x5 = Var(model.t, within=PositiveReals,initialize=Xt5[0]) 
# model.n5 = Var(model.t, within=PositiveReals,initialize=Xt5[1])
# model.q5 = Var(model.t, within=PositiveReals,initialize=Xt5[2])
# model.f5 = Var(model.t, within=PositiveReals,initialize=Xt5[3])
# model.mu1_constr = ConstraintList()



# def F_in1_def(model, t):
#     if t <= tf_N*1:
#         return model.F_in1[t] == float(mu1[0][0])
#     elif tf_N*1 < t <= tf_N*2:
#         return model.F_in1[t] == float(mu1[1][0])
    
#     elif tf_N*2 < t <= tf_N*3:
#         return model.F_in1[t] == float(mu1[2][0])
        
#     # elif tf_N*3 < t <= tf_N*4:
#     #     return m.Fn[t] == Fcn[3]
#     else:
#         return model.F_in1[t] == float(mu1[3][0])
# model.F_in1_constr = Constraint(model.t, rule=F_in1_def)

# def I1_def(model, t):
#     if t <= tf_N*1:
#         return model.I1[t] == float(mu1[0][1])
#     elif tf_N*1 < t <= tf_N*2:
#         return model.I1[t] == float(mu1[1][1])
    
#     elif tf_N*2 < t <= tf_N*3:
#         return model.I1[t] == float(mu1[2][1])
        
#     # elif tf_N*3 < t <= tf_N*4:
#     #     return m.Fn[t] == Fcn[3]
#     else:
#         return model.I1[t] == float(mu1[3][1])
# model.I1_constr = Constraint(model.t, rule=I1_def)



# defining parameters to be determined
model.I = RangeSet(no_euron) # define number of neurons
model.J = RangeSet(3) # number of inputs
model.k = RangeSet(1)
init_dict_w1 = {(1,1):np.random.normal(0, 1)}
init_dict_b1 = {(1,1):np.random.normal(0, 1)}
init_dict_w2 = {(1,1):np.random.normal(0, 1)}
# init_dict_b2 = {(1,1):0}



for i in range(1,no_euron+1): # range of the number of neuron
    for j in range(1,4):# range of the number of inputs
        # print (i,j)
        init_dict_ij = {(i,j):np.random.normal(0, 0.01)}
        init_dict_w1.update(init_dict_ij)
model.w1 = Var(model.I,model.J, initialize=init_dict_w1,bounds = (-0.1,0.1))

for i in range(1,2): # for 1 layer this is fixed
    for j in range(1,no_euron+1): # range of the number of neuron
        # print (i,j)
        init_dict_ij = {(i,j):np.random.normal(0, 0.01)}
        init_dict_b1.update(init_dict_ij)
model.b1 = Var(model.k,model.I,initialize=init_dict_b1,bounds = (-0.1,0.1))

for i in range(1,2):# for 1 layer this is fixed
    for j in range(1,no_euron+1):# range of the number of neuron
        # print (i,j)
        init_dict_ij = {(i,j):np.random.normal(0, 0.01)}
        init_dict_w2.update(init_dict_ij)
model.w2           = Var(model.k,model.I,initialize=init_dict_w2,bounds = (-0.1,0.1))
model.b2           = Var(initialize=0,bounds = (-0.1,0.01))
model.node1 = Var(model.k,model.I,model.t,initialize=0.1,bounds = (-0.1,0.1))


model.ud               = Var(domain = Reals, bounds=(0,0.1),  initialize=0)
model.un               = Var(domain = Reals, bounds=(0,5),  initialize=2.8)
model.kn               = Var(domain = Reals, bounds=(0,2),  initialize=1.4)
model.theta            = Var(domain = Reals, bounds=(5,10),  initialize=7.5)
model.gamma            = Var(domain = Reals, bounds=(5,10),  initialize=8.4)
model.epsilon          = Var(domain = Reals, bounds=(0,1),  initialize=0.1)


# Define u
model.u1     = Var(model.t,domain = NonNegativeReals, bounds=(0,0.1), initialize=0.001)



# defining derivatives
model.x1dot = DerivativeVar(model.x1, wrt=model.t,domain = Reals,initialize = dXdt1[0])
model.n1dot = DerivativeVar(model.n1, wrt=model.t,domain = Reals,initialize = dXdt1[1])
model.q1dot = DerivativeVar(model.q1, wrt=model.t,domain = Reals,initialize = dXdt1[2])
model.f1dot = DerivativeVar(model.f1, wrt=model.t,domain = Reals,initialize = dXdt1[3])
# model.V1dot = DerivativeVar(model.V1, wrt=model.t,domain = Reals,initialize = dXdt1[4])
# differential equation for u, x, n#

# EXP1 ------------------------------------
def NN_node11_exp1(model,t):

    return model.node1[1,1,t] == tanh(model.x1[t]/x_max*model.w1[1,1]+model.q1[t]/q_max*model.w1[1,2]+ model.I1/I_max *model.w1[1,3] +model.b1[1,1])
model.NN_node11_exp1_cons = Constraint(model.t, rule = NN_node11_exp1)

# def NN_node12_exp1(model,t):

#     return model.node1[1,2,t] == tanh(model.x1[t]/x_max*model.w1[2,1]+model.q1[t]/q_max*model.w1[2,2]+ model.I1/I_max *model.w1[2,3] +model.b1[1,2])
# model.NN_node12_exp1_cons = Constraint(model.t, rule = NN_node12_exp1)

# def NN_node13_exp1(model,t):
#     return model.node1[1,3,t] == tanh(model.x1[t]/x_max*model.w1[3,1]+model.q1[t]/q_max*model.w1[3,2]+ model.I1/I_max *model.w1[3,3] + model.b1[1,3])

# model.NN_node13_exp1_cons = Constraint(model.t, rule = NN_node13_exp1)

# def NN_node14_exp1(model,t):
#     return model.node1[1,4,t] == tanh(model.x1[t]/x_max*model.w1[4,1]+model.q1[t]/q_max*model.w1[4,2]+ model.I1/I_max *model.w1[4,3] + model.b1[1,4])

# model.NN_node14_exp1_cons = Constraint(model.t, rule = NN_node14_exp1)

# def NN_node15_exp1(model,t):
#     return model.node1[1,5,t] == tanh(model.x1[t]/x_max*model.w1[5,1]+model.q1[t]/q_max*model.w1[5,2]+ model.I1/I_max *model.w1[5,3] + model.b1[1,5])

# model.NN_node15_exp1_cons = Constraint(model.t, rule = NN_node15_exp1)

# def NN_node16_exp1(model,t):
#     return model.node1[1,6,t] == tanh(model.x1[t]/x_max*model.w1[6,1]+model.q1[t]/q_max*model.w1[6,2]+ model.I1/I_max *model.w1[6,3] + model.b1[1,6])

# model.NN_node16_exp1_cons = Constraint(model.t, rule = NN_node16_exp1)

# def NN_node17_exp1(model,t):
#     return model.node1[1,7,t] == tanh(model.x1[t]/x_max*model.w1[7,1]+model.q1[t]/q_max*model.w1[7,2]+ model.I1/I_max *model.w1[7,3] + model.b1[1,7])

# model.NN_node17_exp1_cons = Constraint(model.t, rule = NN_node17_exp1)



# def u1_con_exp1(model,t):
#     return model.u1[t] == model.node1[1,1,t]*model.w2[1,1] + model.node1[1,2,t]*model.w2[1,2] + model.node1[1,3,t]*model.w2[1,3] + model.node1[1,4,t]*model.w2[1,4] + model.node1[1,5,t]*model.w2[1,5] + model.node1[1,6,t]*model.w2[1,6] + model.node1[1,7,t]*model.w2[1,7] + model.b2
# model.u1_con_exp1  =Constraint(model.t,rule = u1_con_exp1)

def u1_con_exp1(model,t):
    return model.u1[t] == model.node1[1,1,t]*model.w2[1,1] + model.b2
model.u1_con_exp1  =Constraint(model.t,rule = u1_con_exp1)

# def u1_e1_const(model, t):
#   return model.u1[t] >= 0
# model.u1_e1_const = Constraint(model.t, rule = u1_e1_const)



def dxdt1(model,t):
    if t == 0:
        return Constraint.Skip
    return model.x1dot[t] == model.u1[t] * model.x1[t] - model.ud * model.x1[t]
model.dxdtcon1 = Constraint(model.t, rule = dxdt1)

# def dndt1(model,t):
#     if t == 0:
#         return Constraint.Skip
#     return model.n1dot[t] == -model.un*(model.n1[t]/(model.n1[t]+model.kn))*model.x1[t] + model.F_in1
# model.dndtcon1 = Constraint(model.t, rule = dndt1)

def dndt1(model,t):
    if t == 0:
        return Constraint.Skip
    return model.n1dot[t] == -model.un*(model.n1[t]/(model.n1[t]+model.kn))*model.x1[t]
model.dndtcon1 = Constraint(model.t, rule = dndt1)

def dqdt1(model,t):
    if t == 0:
        return Constraint.Skip
    return model.q1dot[t] == model.un*(model.n1[t]/(model.n1[t]+model.kn)) - (model.u1[t]-model.ud)*model.q1[t]
model.dqdtcon = Constraint(model.t, rule = dqdt1)

def dfdt1(model,t):
    if t == 0:
        return Constraint.Skip
    return model.f1dot[t] == model.u1[t]*(model.theta*model.q1[t]-model.epsilon*model.f1[t]) - model.gamma*model.un*(model.n1[t]/(model.n1[t]+model.kn)) + model.ud*model.epsilon*model.f1[t]
model.dfdtcon1 = Constraint(model.t, rule = dfdt1)


number_datapoints1 = xobs1.shape[1]
number_datapoints2 = xobs2.shape[1]
number_datapoints3 = xobs3.shape[1]
number_datapoints4 = xobs4.shape[1]


number_spc1 = xobs1.shape[0]
number_spc2 = xobs2.shape[0]
number_spc3 = xobs3.shape[0]
number_spc4 = xobs4.shape[0]
 

# def obj(model):

 

#     # variance1    = (sum((model.x1[t]-model.x1_noise[t])**2 for t in model.tm)+sum((model.n1[t]-model.n1_noise[t])**2 for t in model.tm)+sum((model.q1[t]-model.q1_noise[t])**2 for t in model.tm)+sum((model.f1[t]-model.f1_noise[t])**2 for t in model.tm))/(number_datapoints1 * number_spc1)


 

#     # obj1 = (sum((model.x1[t]-model.x1_noise[t])**2 for t in model.tm)+sum((model.n1[t]-model.n1_noise[t])**2 for t in model.tm)+sum((model.q1[t]-model.q1_noise[t])**2 for t in model.tm)+sum((model.f1[t]-model.f1_noise[t])**2 for t in model.tm))/2/variance1 - (number_datapoints1 * number_spc1)*log(1/(sqrt(2*3.14159*variance1)))

#     obj1 = (sum((model.x1[t]-model.x1_noise[t])**2 for t in model.tm)+sum((model.n1[t]-model.n1_noise[t])**2 for t in model.tm)+sum((model.q1[t]-model.q1_noise[t])**2 for t in model.tm)+sum((model.f1[t]-model.f1_noise[t])**2 for t in model.tm))/2


#     return obj1

#     # return variance1+variance2+variance3

# model.obj = Objective(rule=obj)


def obj(model):
    return 1/2*(sum(((model.x1[t]-model.x1_noise[t])**2) +((model.n1[t]-model.n1_noise[t])**2) for t in model.tm))
model.obj = Objective(rule=obj)


# -- model display -- #

# -- creating optimization problem -- #
instance = model.create_instance(data_init)
instance.x1[0].fix(0.18)
instance.n1[0].fix(N1)
instance.q1[0].fix(80)
instance.f1[0].fix(120)
# instance.V1[0].fix(0.5)

discretizer = TransformationFactory('dae.collocation')
discretizer.apply_to(instance,nfe=60,ncp=3,wrt=instance.t,scheme='LAGRANGE-RADAU')


    # fix initial value

solver=SolverFactory('ipopt')
# solver.options['max_iter'] = 100000
# solver.options['tol'] = 1e-9
# solver.options['print_level'] = 5  # Adjust based on your needs
results = solver.solve(instance, tee=True, logfile='solver_log.txt')

WARNING (W1002): Setting Var 'w1[2,2]' to a numeric value
    `-0.19643452754352234` outside the bounds (-0.1, 0.1).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'w1[3,3]' to a numeric value
    `-0.1432986689257404` outside the bounds (-0.1, 0.1).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'w1[5,1]' to a numeric value
    `-0.1599762077415162` outside the bounds (-0.1, 0.1).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'w1[7,1]' to a numeric value
    `0.16390077101218145` outside the bounds (-0.1, 0.1).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'w1[7,3]' to a numeric value
    `0.12311416605929695` outside the bounds (-0.1, 0.1).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'b1[1,1]' to a numeric value `0.14377123

WARNING (W1002): Setting Var 'w2[1,4]' to a numeric value
    `-0.11252566204428792` outside the bounds (-0.1, 0.1).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
Ipopt 3.12.13: ERROR: Solver (ipopt) returned non-zero return code (3221226505)
ERROR: See the solver log above for diagnostic information.


ApplicationError: Solver (ipopt) did not exit normally